In [1]:
!pip install accelerate peft bitsandbytes transformers 


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import os
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    pipeline,
    logging,
)


In [2]:
import torch
print(torch.cuda.is_available())

True


INTEGRATION WITH **RAG**

In [4]:
!pip install langchain faiss-gpu sentence_transformers
!pip install langchain-community

ERROR: Could not find a version that satisfies the requirement faiss-gpu (from versions: none)
ERROR: No matching distribution found for faiss-gpu

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ------ --------------------------------- 0.4/2.2 MB 23.7 MB/s eta 0:00:01
   ------------------ --------------------- 1.0/2.2 MB 16.4 MB/s eta 0:00:01
   ----------------------------------- ---- 2.0/2.2 MB 15.6 MB/s eta 0:00:01
   ---------------------------------------- 2.2/2.2 MB 14.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/983.6 kB ? eta -:--:--
   --------------------------------------  983.0/983.6 kB 31.4 MB/s eta 0:00:01
   --------------------------------------- 983.6/983.6 kB 20.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/357.9 kB ? eta -:--:--
   --------------------------------------- 357.9/357.9 kB 23.2 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.2.6
    Uninstalling langchain-core-0.2.6:
      Successfully uninstalled langchain-core-0.2.6
  Attempting uninstall: langchain
    Found existing inst


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFacePipeline

In [2]:
from transformers import AutoTokenizer
from optimum.intel.openvino import OVModelForCausalLM

ov_model_name='HannahJohn/openvino_llama2-chat_finetuned'

ov_model = OVModelForCausalLM.from_pretrained(ov_model_name,device='GPU')
tokenizer = AutoTokenizer.from_pretrained(ov_model_name)

INFO:nncf:NNCF initialized successfully. Supported frameworks detected: torch, onnx, openvino


c:\Users\hp\openvino_env\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Compiling the model to GPU ...
Setting OpenVINO CACHE_DIR to C:\Users\hp\.cache\huggingface\hub\models--HannahJohn--openvino_llama2-chat_finetuned\snapshots\677a2b05573d04c339bd488c55fac2296a7fb91e\model_cache


In [3]:
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.prompts import ChatPromptTemplate, PromptTemplate, MessagesPlaceholder
from langchain_core.messages import AIMessage, HumanMessage
from langchain.chains import ConversationalRetrievalChain

In [4]:
from transformers import pipeline

In [42]:
model_name = 'sentence-transformers/all-MiniLM-L6-v2'
model_kwargs = {'device': 'cuda'}


embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)

DB_FAISS_PATH = 'vectorstores/db_faiss'

custom_prompt_template = """[INST] <<SYS>>
Use the following pieces of information to answer the user's question.
<</SYS>>

Context:
{context}

Question:
{question}

Return the helpful answer below.
[/INST]

Helpful answer:

"""

def set_custom_prompt():
    """
    Prompt template for QA retrieval for each vector store
    """
    prompt = PromptTemplate(template=custom_prompt_template, input_variables=['context', 'question'])
    return prompt

def create_llm(model, tokenizer):
    # Create the Hugging Face text generation pipeline
    text_generation_pipeline = pipeline(
    "text-generation",
    model=model,
    max_new_tokens=256,
    #max_length=300,
    tokenizer=tokenizer, # Adjust as needed for desired length
    #do_sample=True,   # Enable sampling for more natural text
    # Control randomness (higher = more creative)
    #top_p=0.95,       # Nucleus sampling for better quality
    #no_repeat_ngram_size=2, # Prevent repetition of phrases
    #early_stopping=True, # Stop when model thinks it's done
    pad_token_id=tokenizer.eos_token_id # Replace pad tokens with EOS
    )

    # Wrap the pipeline in LangChain's HuggingFacePipeline
    llm = HuggingFacePipeline(pipeline=text_generation_pipeline)
    return llm

def retrieval_qa_chain(llm, prompt, db):
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type='stuff',
        retriever=db.as_retriever(search_kwargs={'k': 2}),
        return_source_documents=True,
        chain_type_kwargs={'prompt': prompt},
    )
    return qa_chain

def qa_bot(model, tokenizer):
    embeddings = HuggingFaceEmbeddings(model_name=model_name,model_kwargs=model_kwargs)
    db = FAISS.load_local(DB_FAISS_PATH, embeddings,allow_dangerous_deserialization=True)
    llm = create_llm(model, tokenizer)
    qa_prompt = set_custom_prompt()
    qa = retrieval_qa_chain(llm, qa_prompt, db)
    return qa,db

def final_result(query, model, tokenizer):
    qa,db = qa_bot(model, tokenizer)

    search_scores = db.similarity_search_with_score(query)
    score_info = "\n".join([f"Doc: {doc.page_content}, Score: {score}" for doc, score in search_scores])

    qa_result = qa({'query': query})

    response = qa_result['result']


    helpful_answer = response.split('Helpful answer:')[-1].strip()

    return f"{helpful_answer.strip()}\n\nSimilarity Scores:\n{score_info}"

c:\Users\hp\openvino_env\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [44]:
query = "What should I do if I am addicted to vapeing?"
result = final_result(query, ov_model, tokenizer)
print(result)

c:\Users\hp\openvino_env\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


If you are addicted to vaping, there are several things you can do to help you stop. First, it is important to understand that nicotine is an addictive drug and can be difficult to stop using. However, there are many resources available to help you quit. Nicotine replacement therapy (NRT) can be helpful in reducing withdrawal symptoms. You can also try prescription medications such as bupropion or varenicline. Another option is to switch to a new habit, such as chewing gum or sucking on a candy. It is also important to be aware of the triggers that cause you to want to vape and to avoid them. Finally, it is important to be patient with yourself and to give yourself time to recover. It can take several months to a year to fully recover from nicotine addiction. If you are having trouble quitting, you can also try talking to a counselor or a support group. Remember, you can do it! You can break the habit and live a healthier life. Good luck!

Similarity Scores:
Doc: Nichols, M. P„ 242
Nic

In [45]:
query = "I've tried quitting before but failed. What are some strategies for staying clean?"
result = final_result(query, ov_model, tokenizer)
print(result)

c:\Users\hp\openvino_env\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


I'm sorry to hear you've tried quitting before and failed. It's important to remember that recovery is a journey and not a destination. It's important to be patient with yourself and to not give up. There are many strategies that can help you stay clean. Here are some suggestions:

1. Find a support group. Being around people who are going through similar experiences can be very helpful.
2. Find a therapist. A therapist can help you identify the triggers that are causing you to relapse and help you develop coping skills.
3. Find healthy alternatives. If you are using something to cope with stress or other emotions, find healthy alternatives. For example, if you are using alcohol to cope with stress, find healthy ways to cope with stress such as exercise, meditation, or talking to a friend.
4. Be kind to yourself. It's important to remember that you are not a failure. You are not alone and you can do it.
5. Take it one day at a time. Recovery is a journey and it's important to take it o

In [46]:
query = "How do I forgive myself for the mistakes I've made because of my addiction?"
result = final_result(query, ov_model, tokenizer)
print(result)

c:\Users\hp\openvino_env\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Forgiving yourself for mistakes you've made because of your addiction can be a challenging process. It's important to remember that you are not alone in this process and that there are many resources available to help you.

One way to start the process of forgiveness is to acknowledge the mistakes you've made and to accept them as part of your journey. It's important to remember that you are not defined by your mistakes, but rather by your ability to learn from them and grow.

Another way to forgive yourself is to practice self-compassion. This means treating yourself with kindness, understanding, and acceptance. It's important to remember that you are not perfect and that you are doing the best you can.

It's also important to remember that you are not alone in this process. There are many resources available to help you work through your addiction and to forgive yourself. Consider reaching out to a therapist or support group for help.

Remember, forgiveness is a process and it takes 